In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import pageviewapi
import datetime as dt
from bs4 import BeautifulSoup
import altair as alt

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [5]:
import numpy as np

---

### data from the national council of state legislatures

In [6]:
# url = "https://www.ncsl.org/research/civil-and-criminal-justice/clearing-criminal-records-for-cannabis-offenses.aspx"

In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup


html = urlopen(
    "https://www.ncsl.org/research/civil-and-criminal-justice/clearing-criminal-records-for-cannabis-offenses.aspx"
)

In [8]:
soup = BeautifulSoup(html, "html.parser")

In [9]:
states = soup.find_all("div", class_="notshowing")

In [10]:
names = []
record_clearing = []
cannabis_specific = []
citation = []
proposed_law = []
recreational_use = []
medical_use = []

for state in states[0:51]:
    names.append(state.find("h4").text)
    data = state.find_all("p")
    record_clearing.append(data[0].text)
    if "Specific" in data[1].text:
        cannabis_specific.append(data[1].text)
        citation.append(data[2].text)
    else:
        cannabis_specific.append("NaN")
        citation.append(data[1].text)
    if "Proposed" in data[2].text:
        proposed_law.append(data[2].text)
    else:
        if "Proposed" in data[3].text:
            proposed_law.append(data[3].text)
        else:
            proposed_law.append("NaN")
    if "Recreational" in data[2].text:
        recreational_use.append(data[2].text)
    else:
        if "Recreational" in data[3].text:
            recreational_use.append(data[3].text)
        else:
            if len(data) > 4:
                if "Recreational" in data[4].text:
                    recreational_use.append(data[4].text)
    if "Medical" in data[3].text:
        medical_use.append(data[3].text)
    else:
        if len(data) > 4:
            if "Medical" in data[4].text:
                medical_use.append(data[4].text)
            else:
                if len(data) > 5:
                    if "Medical" in data[5].text:
                        medical_use.append(data[5].text)


#    for item in data:
#        if "Specific" in data[1].text:
#            cannabis_specific.append(data[1].text)
#            citation.append(data[2].text)
#        else:
#            cannabis_specific.append("NaN")
#            citation.append(data[1])

In [11]:
len(medical_use)

51

In [12]:
df = pd.DataFrame(
    {
        "state": names,
        "record_clearing": record_clearing,
        "cannabis_specific": cannabis_specific,
        "citation": citation,
        "proposed_law": proposed_law,
        "recreational_use": recreational_use,
        "medical_use": medical_use,
    }
)

In [13]:
df["record_clearing"] = df["record_clearing"].str.replace(
    "Cannabis Record Clearing: ", ""
)

df["citation"] = df["citation"].str.replace("Citation: ", "")

df["cannabis_specific"] = df["cannabis_specific"].str.replace("Cannabis Specific: ", "")

df["proposed_law"] = df["proposed_law"].str.replace("Proposed Legislation: ", "")

df["proposed_law"] = df["proposed_law"].str.replace("Proposed Ballot Initiative: ", "")

df["recreational_use"] = df["recreational_use"].str.replace("Recreational Use: ", "")

df["medical_use"] = df["medical_use"].str.replace("Medical Use: ", "")

In [14]:
df

,state,record_clearing,cannabis_specific,citation,proposed_law,recreational_use,medical_use
0,Alabama,No,NaN,Ala. Code 1975 § 15-27-1 et seq.,NaN,No,No
1,Alaska,No,NaN,AS § 12.62.005 et seq.; AS § 22.35.010 et seq.,SB 8,Yes,Yes
2,Arizona,No,NaN,A.R.S. § 13-901 et seq.,HB 2178,No,Yes
3,Arkansas,Yes,No,A.C.A. § 16-90-1401 et seq.,Arkansas Marijuana Expungement Act,No,Yes
4,California,Yes,Yes,West's Ann. Cal. Health & Safety Code § 11357 et seq.,NaN,Yes,Yes
5,Colorado,Yes,No,C.R.S.A. § 24-72-701 et seq.,NaN,Yes,Yes
6,Connecticut,Yes,No,C.G.S.A. § 54-142a et seq.,NaN,Decriminalized,Yes
7,Delaware,Yes,No,11 Del.C. § 4371 et seq.,NaN,Decriminalized,Yes
8,District of Columbia,Yes,No,DC ST § 16-801 et seq.,NaN,Yes; No Sales,Yes
9,Florida,No,NaN,West's F.S.A. § 943.01 et seq.,SB 1342,No,Yes


why doesn't [this infographic](https://ccresourcecenter.org/wp-content/uploads/2021/05/Marijuana-Record-Relief-Infographic_final-scaled.jpg) match up? <br>
analyze and compare data from [here](https://ccresourcecenter.org/state-restoration-profiles/50-state-comparison-marijuana-legalization-expungement/)?


In [15]:
html = urlopen("https://norml.org/laws/expungement/")

In [16]:
soup = BeautifulSoup(html, "html.parser")

In [17]:
state_list = soup.find_all("ul", class_="")

In [18]:
states = state_list[0].find_all("li")

In [19]:
linklist = state_list[0].find_all("a", href=True)

In [20]:
links = []
statename = []
asterisk = []
for item in linklist:
    statename.append(item.text)
    links.append(item["href"])

In [21]:
for state in states:
    asterisk.append(state.text.strip())

In [22]:
count = []
for a in asterisk:
    count.append(a.count("*"))

In [23]:
norml = pd.DataFrame({"state": statename, "law_link": links, "caveats": count})

In [24]:
asterisk = []
for a in norml["caveats"]:
    asterisk.append(a.count("*"))

AttributeError: 'int' object has no attribute 'count'

In [ ]:
conditions = [
    norml["caveats"] == 0,
    norml["caveats"] == 1,
    norml["caveats"] == 2,
    norml["caveats"] == 3,
    norml["caveats"] == 4,
]
values = [
    "NaN",
    "Automatic expungement process for qualifying convictions",
    "Annulment process available for certain offenders",
    "Certain marijuana-related convictions may be sealed",
    "Those with certain convictions may be eligible for unconditional pardons from a state advisory board",
]
norml["caveat_language"] = np.select(conditions, values)

In [ ]:
# definitely out of date--doesn't include arkansas
norml